In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets, models

import sys
sys.path.append('Scripts')
from utils import getData

In [2]:
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
image_size = 224
batch_size = 1
workers = 1
ngpu = 1
validation_split = 0
train_split = 0
shuffle_dataset = True
random_seed = 42
image_size = 224
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# device = "cpu"    

In [5]:
folder = 'Test_GASF'
figPath = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\'+folder+'\\512'
dataloaders, dataset_sizes, dataloader, class_names = getData(figPath, image_size, train_split, validation_split, batch_size = batch_size)

In [6]:
class predictor(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def predict(self, dataloader):
        outputs = []
        targets = []
        for sample, target in dataloader:
            o = torch.argmax(F.softmax(self.model(sample), dim = 1)).item()
            outputs.append(o)
            targets.append(target.item())
            
        outputs = np.array(outputs)
        targets = np.array(targets)
        right_pred = 1*(outputs == targets)
        return outputs, right_pred

In [8]:
method = 'gadf'
path = 'Models\\'+method
names = os.listdir(path)[1:]

In [55]:
modelDict = {
#     'resnet18': models.resnet18(),
#     'resnet34': models.resnet34(),
#     'resnet50': models.resnet50(),
#     'resnet101': models.resnet101(),
#     'resnet152': models.resnet152(),
#     'densenet121': models.densenet121(),
#     'densenet161': models.densenet161(),
#     'densenet169': models.densenet169(),
#     'densenet201': models.densenet201(),
#     'alexnet': models.alexnet(),
    'googlenet': models.googlenet(),
#     'vgg11': models.vgg11(),
#     'vgg13': models.vgg13(),
#     'vgg16': models.vgg16()
}

In [ ]:
modelName = 'googlenet'
network = modelDict[modelName]
modelPath = path+"\\"+modelName+'\\cca128\\model.pth'
network.load_state_dict(torch.load(modelPath))
predic = predictor(network)
_, out = predic.predict(dataloader)
resultsDict[modelName] = out
#     numFeatures = modelDict[modelName][1]

#     temp = modelName
#     for d in modelName:
#         if d.isdigit():
#             temp = temp.replace(d, '')

    

In [61]:
from scipy import stats
stats.ttest_ind(resultsDict['resnet18'], resultsDict['densenet121'], equal_var=True)

Ttest_indResult(statistic=0.0, pvalue=1.0)

In [62]:
np.count_nonzero(resultsDict['resnet18'])/500

0.242

In [63]:
np.count_nonzero(resultsDict['densenet121'])/500

0.242

In [66]:
np.count_nonzero(resultsDict['alexnet'])/500

0.2

In [67]:
np.count_nonzero(resultsDict['densenet201'])/500

0.24